In [1]:
import requests
from bs4 import BeautifulSoup

### 한 페이지 뉴스 기사 크롤링 코드

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_news_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        content_element = soup.find('div', {'id': 'articleBodyContents'})
        if content_element:
            content = content_element.get_text(separator=' ', strip=True)
            return content
    return None

def crawl_naver_news(query, num_pages):
    base_url = "https://search.naver.com/search.naver"
    news_list = []

    for page in range(1, num_pages + 1):
        params = {
            "where": "news",
            "sm": "tab_jum",
            "query": query,
            "start": (page - 1) * 10 + 1
        }

        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            news_titles = soup.select(".news_tit")
            news_urls = soup.select(".news_tit a")

            for title, url in zip(news_titles, news_urls):
                news_content = scrape_news_content(url['href'])
                news_list.append({"제목": title.get_text(strip=True), "내용": news_content})
        else:
            print(f"페이지 {page}의 뉴스를 가져오는 데 실패했습니다.")

    df = pd.DataFrame(news_list)
    df.to_excel("naver_news.xlsx", index=False)
    print(f"{num_pages}페이지의 뉴스를 엑셀 파일로 저장했습니다.")

# 크롤링 실행
crawl_naver_news("+전자담배 아이코스 후기", 8)


8페이지의 뉴스를 엑셀 파일로 저장했습니다.


### 페이지마다 이동해서 내용수집 - 네이버뉴스만

In [7]:
import requests
from bs4 import BeautifulSoup
import time
import pyautogui
from openpyxl import Workbook
from openpyxl.styles import Alignment

# user input
keyword = pyautogui.prompt("keyword")
lastpage = int(pyautogui.prompt("page"))

# create xlsx file
wb = Workbook()
ws = wb.create_sheet(keyword)
ws.column_dimensions['A'].width = 60
ws.column_dimensions['B'].width = 60
ws.column_dimensions['C'].width = 120

# variables
row = 1

for page_num in range(1, lastpage + 1):
    print(f"{page_num} page...")
    start_index = (page_num - 1) * 10
    response = requests.get(f"https://search.naver.com/search.naver?where=news&sm=tab_jum&query={keyword}&start={start_index}")
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    articles = soup.select("div.info_group")

    for article in articles:
        links = article.select("a.info")
        if len(links) >= 2:
            url = links[1].attrs["href"]
            response = requests.get(url, headers={'User-agent': 'Mozila/5.0'})  # to avoid error, use headers
            html = response.text  # for each url get html
            soup = BeautifulSoup(html, "html.parser")  # for each html make soup

            # separation
            if "entertain" in response.url:  # to avoid redirection error
                title = soup.select_one(".end_tit")  # get the title
                content = soup.select_one("#articeBody")  # get the body

            elif "sports" in response.url:  # to avoid redirection error
                title = soup.select_one("h4.title")  # get the title
                content = soup.select_one("#newsEndContents")  # get the body

                # delete unnecessary elements
                divs = content.select("div")
                for div in divs:
                    div.decompose()

                paragraphs = content.select("p")
                for p in paragraphs:
                    p.decompose()

            else:
                title = soup.select_one(".media_end_head_headline")  # get the title
                content = soup.select_one("#dic_area")  # get the body

            print("========LINK========\n", url)
            print("========TITLE========\n", title.text.strip())
            print("========BODY========\n", content.text.strip())

            # write datas in xlsx file
            ws[f"A{row}"] = url
            ws[f"B{row}"] = title.text.strip()
            ws[f"C{row}"] = content.text.strip()

            ws[f"C{row}"].alignment = Alignment(wrap_text=True)  # line breaking

            row += 1
            time.sleep(0.3)

    wb.save(f"{keyword}_result.xlsx")

1 page...
========LINK========
 https://n.news.naver.com/mnews/article/018/0003869195?sid=101
========TITLE========
 [강신우의 닥치Go]전자담배 ‘아이코스’ 2주 사용후기
========BODY========
 - ‘쑥을 찌는 향’이 나는 연기- 일반담배 생각 ‘싹’ 사라져- 홀더충전, 청소는 다소 불편



‘아이코스’ 전용 궐련형 담배 히츠. 히츠 그린과 블루는 인기가 많아 동네 편의점 매대에 재고가 떨어진 모습. 이데일리DB[이데일리 강신우 기자] 어느 날 갑자기 전자담배 ‘아이코스’를 들고온 회사 동료. 냄새도 안나고 일반 담배 생각이 아예 없어졌다기에 궁금해서 직접 구입·사용해 봤다. 지금으로부터 2주 전. 편의점 CU를 찾았다. “아이코스 주세요”라고 했더니 이름과 기기 색상(화이트/네이비)을 적고 기다리라고 했다. 그것도 무기한. 그런데 3일 만에 받았다. 가격은 권장소비자가격 12만원인데 할인가에 판다. 가격은 9만7000원. 



전자담배 ‘아이코스’와 권렬형 담배 ‘히츠’. 이데일리DB일반담배와 비슷한 아이코스 전용담배인 ‘히츠’(니코틴0.5g, 포름알데히드와 같은 발암물질도 포함돼 있음·20개입·4300원)도 함께 샀다. 구수한향인데 향이나 연기가 약한 것은 ‘실버’ 강한 것은 ‘엠버’, 후레쉬향인데 약한 것은 ‘그린’ 강한 것은 ‘블루’. 이렇게 종류는 총 4가지다. 처음이니까 ‘실버’를 샀다. 제품을 샀다면 충전해야 한다. USB 충전방식으로 포켓 충전기 겉에 있는 4개의 표시등에 흰색등이 모두 켜져 있으면 충전이 완료된 것. 이후 담배모양의 기기(홀더)를 충전기 안에 넣으면 맨 위에 있는 흰색 표시등이 깜빡이면서 충전을 시작한다. 깜빡임이 멈추면 충전이 끝난 것이다. 대략 4분이 걸린다. 



아이코스 ‘포켓 충전기’. 이데일리DB이제 홀더 구멍에 히츠를 꾹 눌러 넣고 홀더 중앙의 히팅버튼을 누른다. 진동이 오면 손을 떼고 10초 정도 기다

### 네이버 뉴스가 아닌 모든 네이버 뉴스 본문 크롤링 코드